# Atividade 2: Aprendizado Auto-Supervisionado com Autoencoder
## MNIST Dataset

Este notebook demonstra o uso de autoencoders para aprendizado não supervisionado usando o dataset MNIST.

In [ ]:
# Importar bibliotecas necessárias
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model

print(f"TensorFlow version: {tf.__version__}")

## 1. Carregar e Preparar os Dados

In [ ]:
# Carregar dataset MNIST
(x_train, _), (x_test, _) = mnist.load_data()

# Normalizar os dados para o intervalo [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Achatar as imagens (28x28 -> 784)
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

print(f"Shape dos dados de treino: {x_train.shape}")
print(f"Shape dos dados de teste: {x_test.shape}")

## 2. Definir a Arquitetura do Autoencoder

In [ ]:
# Dimensões
input_dim = 784
encoding_dim = 32  # Dimensão do espaço latente (compressão)

# Encoder
input_img = keras.Input(shape=(input_dim,))
encoded = layers.Dense(128, activation='relu')(input_img)
encoded = layers.Dense(64, activation='relu')(encoded)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)

# Decoder
decoded = layers.Dense(64, activation='relu')(encoded)
decoded = layers.Dense(128, activation='relu')(decoded)
decoded = layers.Dense(input_dim, activation='sigmoid')(decoded)

# Autoencoder completo
autoencoder = Model(input_img, decoded)

# Modelo do Encoder separado
encoder = Model(input_img, encoded)

# Visualizar a arquitetura
autoencoder.summary()

## 3. Compilar e Treinar o Modelo

In [ ]:
# Compilar o modelo
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o autoencoder
history = autoencoder.fit(
    x_train, x_train,
    epochs=50,
    batch_size=256,
    shuffle=True,
    validation_data=(x_test, x_test),
    verbose=1
)

## 4. Visualizar o Histórico de Treinamento

In [ ]:
# Plotar a perda durante o treinamento
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda do Modelo')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia do Modelo')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 5. Avaliar o Autoencoder - Visualizar Reconstruções

In [ ]:
# Fazer predições
decoded_imgs = autoencoder.predict(x_test)

# Visualizar imagens originais e reconstruídas
n = 10  # Número de imagens a visualizar
plt.figure(figsize=(20, 4))

for i in range(n):
    # Imagem original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
    plt.title("Original")
    plt.axis('off')
    
    # Imagem reconstruída
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28), cmap='gray')
    plt.title("Reconstruída")
    plt.axis('off')

plt.tight_layout()
plt.show()

## 6. Visualizar o Espaço Latente

In [ ]:
# Codificar os dados de teste
(x_train_full, y_train_full), (x_test_full, y_test_full) = mnist.load_data()
x_test_normalized = x_test_full.astype('float32') / 255.0
x_test_flat = x_test_normalized.reshape((len(x_test_normalized), np.prod(x_test_normalized.shape[1:])))

encoded_imgs = encoder.predict(x_test_flat)

# Usar PCA para reduzir para 2D se encoding_dim > 2
if encoding_dim > 2:
    from sklearn.decomposition import PCA
    pca = PCA(n_components=2)
    encoded_2d = pca.fit_transform(encoded_imgs)
else:
    encoded_2d = encoded_imgs

# Plotar o espaço latente colorido por classe
plt.figure(figsize=(10, 8))
scatter = plt.scatter(encoded_2d[:, 0], encoded_2d[:, 1], c=y_test_full, cmap='tab10', alpha=0.6, s=5)
plt.colorbar(scatter, label='Dígito')
plt.title('Visualização do Espaço Latente (2D)')
plt.xlabel('Dimensão 1')
plt.ylabel('Dimensão 2')
plt.grid(True, alpha=0.3)
plt.show()

## 7. Detectar Anomalias

Autoencoders podem ser usados para detecção de anomalias. Imagens com alto erro de reconstrução podem ser consideradas anomalias.

In [ ]:
# Calcular erro de reconstrução (MSE)
mse = np.mean(np.square(x_test - decoded_imgs), axis=1)

# Plotar distribuição dos erros
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(mse, bins=50, alpha=0.7, color='blue')
plt.xlabel('Erro de Reconstrução (MSE)')
plt.ylabel('Frequência')
plt.title('Distribuição do Erro de Reconstrução')
plt.grid(True, alpha=0.3)

# Identificar anomalias (top 1% de erros)
threshold = np.percentile(mse, 99)
anomalies = np.where(mse > threshold)[0]

plt.subplot(1, 2, 2)
plt.hist(mse, bins=50, alpha=0.7, color='blue', label='Normal')
plt.axvline(threshold, color='red', linestyle='--', linewidth=2, label=f'Limiar (99%): {threshold:.4f}')
plt.xlabel('Erro de Reconstrução (MSE)')
plt.ylabel('Frequência')
plt.title('Detecção de Anomalias')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nNúmero de anomalias detectadas: {len(anomalies)}")

In [ ]:
# Visualizar algumas anomalias
n_anomalies = min(10, len(anomalies))
plt.figure(figsize=(20, 4))

for i in range(n_anomalies):
    idx = anomalies[i]
    
    # Imagem original
    ax = plt.subplot(2, n_anomalies, i + 1)
    plt.imshow(x_test[idx].reshape(28, 28), cmap='gray')
    plt.title(f"MSE: {mse[idx]:.4f}")
    plt.axis('off')
    
    # Imagem reconstruída
    ax = plt.subplot(2, n_anomalies, i + 1 + n_anomalies)
    plt.imshow(decoded_imgs[idx].reshape(28, 28), cmap='gray')
    plt.title("Reconstruída")
    plt.axis('off')

plt.suptitle('Anomalias Detectadas (Top 10)', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

## 8. Conclusões

Neste notebook, demonstramos:

1. **Construção de um Autoencoder**: Criamos uma rede neural que aprende a comprimir e reconstruir imagens MNIST
2. **Aprendizado Não Supervisionado**: O modelo aprendeu representações úteis sem usar labels
3. **Redução de Dimensionalidade**: Comprimimos imagens de 784 dimensões para 32 dimensões
4. **Visualização do Espaço Latente**: Observamos como o modelo organiza os dígitos no espaço latente
5. **Detecção de Anomalias**: Usamos o erro de reconstrução para identificar imagens anômalas

### Possíveis Extensões:
- Experimentar com diferentes arquiteturas (Convolutional Autoencoder, Variational Autoencoder)
- Ajustar a dimensão do espaço latente
- Aplicar a outros datasets
- Usar o autoencoder para pré-treinamento em tarefas supervisionadas